In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [2]:
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
print( X_train.shape )
X_train[0][:10, :10]

(60000, 28, 28)


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36],
       [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253],
       [  0,   0,   0,   0,   0,   0,   0,  18, 219, 253],
       [  0,   0,   0,   0,   0,   0,   0,   0,  80, 156]], dtype=uint8)

In [4]:
X_train = X_train / 255
X_test = X_test / 255

In [5]:
X_train.max(), X_train.min()

(1.0, 0.0)

In [6]:
model = tf_keras.models.Sequential([
  tf_keras.layers.Input(shape=(28, 28, 1)),
  tf_keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal"),
  tf_keras.layers.MaxPool2D(pool_size=2),
  tf_keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal"),
  tf_keras.layers.MaxPool2D(pool_size=2),
  tf_keras.layers.Flatten(),
  tf_keras.layers.Dense(units=128, activation="relu", kernel_initializer="he_normal"),
  tf_keras.layers.Dense(units=10, activation="softmax")
])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [7]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [8]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5


750/750 [==============================] - 13s 16ms/step - loss: 0.1458 - accuracy: 0.9556 - val_loss: 0.0513 - val_accuracy: 0.9852
Epoch 2/5
750/750 [==============================] - 12s 16ms/step - loss: 0.0443 - accuracy: 0.9857 - val_loss: 0.0440 - val_accuracy: 0.9862
Epoch 3/5
750/750 [==============================] - 12s 16ms/step - loss: 0.0295 - accuracy: 0.9904 - val_loss: 0.0402 - val_accuracy: 0.9875
Epoch 4/5
750/750 [==============================] - 12s 16ms/step - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.0449 - val_accuracy: 0.9868
Epoch 5/5
750/750 [==============================] - 12s 16ms/step - loss: 0.0151 - accuracy: 0.9953 - val_loss: 0.0420 - val_accuracy: 0.9894


In [9]:
model.save("models/mnist-cnn-model.keras")

In [10]:
model2 = tf_keras.models.load_model('models/mnist-cnn-model.keras')

In [11]:
model2.evaluate(X_train, y_train)

1875/1875 [==============================] - 5s 3ms/step - loss: 0.0151 - accuracy: 0.9957


[0.015116048976778984, 0.9957166910171509]

In [14]:
model2.predict(X_test[:1]).argmax(axis=1)[0]

1/1 [==============================] - 0s 20ms/step


7

In [ ]:
# !pip install opencv-python

In [15]:
import cv2
cv2.__version__

'4.9.0'

In [29]:
cap = cv2.VideoCapture(0) # 컴퓨터에 연결된 카메라에 연결하는 명령
if cap.isOpened():
    # print ("카메라가 연결되었습니다.")
    while True:
        ret, img = cap.read() # 한 프레임 읽기
        if ret: # 정상적으로 프레임을 읽은 경우

            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 흑백이미지로 변환 (회색조 포함)
            thresh, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV) # 검은색/흰색만 사용하는 이미지로 변경
            contours, hierarchy = \
                cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # 이미지 안의 사물을 찾아서 경계선 정보 반환

            try:
                for idx in range(len(contours)):
                    contour = contours[idx]
                    (x,y), radius = cv2.minEnclosingCircle(contour) # 모든 경계선을 포함하는 가장 작은 원 정보 반환                
                    if radius > 10:
                        xs, ys = int(x-radius), int(y-radius)  # 좌측 상단 좌표
                        xe, ye = int(x+radius), int(y+radius)  # 우측 하단 좌표
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1) # 현재 프레임 이미지에 사각형 표시
                        
                        roi = bin_img[ys:ye, xs:xe]
                        dst = cv2.resize(roi, (50, 50), interpolation=cv2.INTER_AREA)
                        dst = cv2.resize(dst, (24, 24), interpolation=cv2.INTER_AREA)
                        A = np.zeros((28, 28))
                        A[2:-2, 2:-2] = dst[:, :]
                        A = A.reshape(-1, 28, 28)
                        predicted_value = model2.predict(A)
                        num = predicted_value.argmax(axis=1)[0]
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 1, (200, 0, 0))

                # cv2.imshow("Image", img) # 윈도우를 열고 윈도우에 이미지 출력
                # cv2.imshow("Image", g_img)
            except:
                pass

            cv2.imshow("Image", bin_img)

            # waitKey : 키보드 입력 대기
            if cv2.waitKey(1)&0xFF == 27: # ESC 키 입력 여부 확인
                break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step
